difference from mapping cache, between revised and stage2

In [1]:
# NOTE: inherit kernel from "10-pubtator-results.ipynb"

In [100]:
set_b = set(b_mappings["mapping_id"].tolist())
set_bb = set(bb_mappings["mapping_id"].tolist())
diff_b_bb = set_b.difference(set_bb)
diff_bb_b = set_bb.difference(set_b)
print(diff_b_bb)
print(diff_bb_b)

{148, 150, 278, 408, 152, 793, 798, 39, 553, 437, 581, 1094, 82, 467, 858, 736, 737, 865, 355, 114, 248, 380}
{448, 864, 546, 136, 842, 588, 144, 49, 830, 211, 601, 604, 829, 190, 159}


In [226]:
def show_info(
    mapping_id: str,
    ebi_df: pd.DataFrame,
    efo_df: pd.DataFrame,
    ner_res: pd.DataFrame,
    this_mappings: pd.DataFrame = None,
    this_top100: pd.DataFrame = None,
    other_mappings: pd.DataFrame = None,
    other_top100: pd.DataFrame = None,
):
    ebi_row = ebi_df[ebi_df["mapping_id"] == mapping_id]
    if len(ebi_row) == 0:
        return
    # ---- general info
    ebi_label = list(ebi_row["query"])[0]
    truth_efo = {"id": list(ebi_row["full_id"])[0], "label": list(ebi_row["MAPPED_TERM_LABEL"])[0], 
                 "mapping_type": list(ebi_row["MAPPING_TYPE"])[0]}
    truth_efo_idx = efo_df.query(f"efo_id == '{truth_efo['id']}'").index.tolist()[0] + 1
    # print(truth_efo_idx)
    # ---- ents
    ents_ebi = ner_res.query("batch == 'ebi'").query(f"mapping_id == {mapping_id}")[["term", "session_res", "ner_res"]].reset_index(drop=True)
    ents_efo = ner_res.query("batch == 'efo'").query(f"mapping_id == {truth_efo_idx}")[["term", "session_res", "ner_res"]].reset_index(drop=True)
    # ---- mapping df
    this_df_init = this_top100\
        .query(f"mapping_id == {mapping_id}")\
        [["prediction", "score"]]\
        .merge(efo_df.rename(columns={"efo_id": "prediction"}), on=["prediction"])\
        .reset_index(drop=True)\
        .assign(idx=lambda df: df.index.tolist())
    this_df = pd.concat([
        this_df_init.iloc[[0]],
        this_df_init.query(f"prediction == '{truth_efo['id']}'")
    ]).drop_duplicates()[["idx", "efo_label", "score"]]
    other_df_init = other_top100\
        .query(f"mapping_id == {mapping_id}")\
        [["prediction", "score"]]\
        .merge(efo_df.rename(columns={"efo_id": "prediction"}), on=["prediction"])\
        .reset_index(drop=True)\
        .assign(idx=lambda df: df.index.tolist())
    other_df = pd.concat([
        other_df_init.iloc[[0]],
        other_df_init.query(f"prediction == '{truth_efo['id']}'")
    ]).drop_duplicates()[["idx", "efo_label", "score"]]
    print(f"  mapping_id: {mapping_id}, ebi_label: {ebi_label}")
    print(f"  goldstandard efo: {truth_efo}")
    print("\n")
    print("Identified ents: ebi")
    if len(ents_ebi) == 0:
        print("    ", None)
    else:
        print("    ", ents_ebi["session_res"].tolist())
        print("    ", ents_ebi["ner_res"].tolist())
    print("Identified ents: efo")
    if len(ents_efo) == 0:
        print("    ", None)
    else:
        print("    ", ents_efo["session_res"].tolist())
        print("    ", ents_efo["ner_res"].tolist())
    print("\n")
    print(f"this_df\n  {this_df}")
    print(f"other_df\n  {other_df}")
    print("\n------------\n")

In [227]:
MAPPING_ID_LIST = list(diff_bb_b)

for idx, mapping_id in enumerate(MAPPING_ID_LIST):
    print(f"# {idx}")
    show_info(
        mapping_id=mapping_id,
        ebi_df=ebi_df,
        efo_df=efo_df,
        ner_res=ner_res,
        this_top100=top100_bb,
        other_top100=top100_b,
    )

# 0
  mapping_id: 448, ebi_label: hayfever or allergic rhinitis
  goldstandard efo: {'id': 'http://www.ebi.ac.uk/efo/EFO_0003956', 'label': 'seasonal allergic rhinitis', 'mapping_type': 'Exact'}


Identified ents: ebi
     [['00000\t0\t29\thayfever or allergic rhinitis\tDisease\tMESH:D006255']]
     [['Disease_MESH_D006255']]
Identified ents: efo
     [['00000\t0\t26\tseasonal allergic rhinitis\tDisease\tMESH:D006255']]
     [['Disease_MESH_D006255']]


this_df
     idx                   efo_label     score
0    0  seasonal allergic rhinitis  0.880174
other_df
     idx                   efo_label     score
0    0           allergic rhinitis  0.950498
1    1  seasonal allergic rhinitis  0.861206

------------

# 1
  mapping_id: 864, ebi_label: bipolar affective disorder
  goldstandard efo: {'id': 'http://www.ebi.ac.uk/efo/EFO_0000289', 'label': 'bipolar disorder', 'mapping_type': 'Exact'}


Identified ents: ebi
     [['00000\t0\t26\tbipolar affective disorder\tDisease\tMESH:D001714']]
 

In [228]:
MAPPING_ID_LIST = list(diff_b_bb)

for idx, mapping_id in enumerate(MAPPING_ID_LIST):
    print(f"# {idx}")
    show_info(
        mapping_id=mapping_id,
        ebi_df=ebi_df,
        efo_df=efo_df,
        ner_res=ner_res,
        this_top100=top100_b,
        other_top100=top100_bb,
    )

# 0
  mapping_id: 148, ebi_label: chronic renal failure
  goldstandard efo: {'id': 'http://www.ebi.ac.uk/efo/EFO_0003884', 'label': 'chronic kidney disease', 'mapping_type': 'Exact'}


Identified ents: ebi
     None
Identified ents: efo
     [['00000\t0\t22\tchronic kidney disease\tDisease\tMESH:D051436']]
     [['Disease_MESH_D051436']]


this_df
     idx               efo_label     score
0    0  chronic kidney disease  0.600379
other_df
     idx               efo_label     score
0    0     renal insufficiency  0.598127
5    5  chronic kidney disease  0.527863

------------

# 1
  mapping_id: 150, ebi_label: dementia in alzheimer s disease
  goldstandard efo: {'id': 'http://www.ebi.ac.uk/efo/EFO_0000249', 'label': "Alzheimer's disease", 'mapping_type': 'Broad'}


Identified ents: ebi
     [['00000\t0\t8\tdementia\tDisease\tMESH:D003704', '00000\t12\t31\talzheimer s disease\tDisease\tMESH:D000544']]
     [['Disease_MESH_D003704', 'Disease_MESH_D000544']]
Identified ents: efo
     None
